In [1]:
from train import *

In [2]:
from train import get_classes, get_anchors
annotation_path = 'data/input.csv'
log_dir = 'logs/000/'
classes_path = 'model_data/stdogs_classes.txt'
anchors_path = 'model_data/yolo_anchors.txt'
val_split = 0.5

class_names = get_classes(classes_path)
num_classes = len(class_names)
anchors = get_anchors(anchors_path)

input_shape = (416, 416)  # multiple of 32, hw

is_tiny_version = len(anchors) == 6  # default setting


with open(annotation_path) as f:
    lines = f.readlines()
np.random.seed(10101)
np.random.shuffle(lines)
np.random.seed(None)
num_val = int(len(lines) * val_split)
num_train = len(lines) - num_val



In [3]:
import tensorflow as tf
batch_size = 2
train_data = data_generator_wrapper(lines[:num_train], batch_size, input_shape, anchors, num_classes)
eval_data = generator = data_generator_wrapper(lines[num_train:], batch_size, input_shape, anchors, num_classes)


W0525 18:44:20.961562 140735679218560 deprecation.py:323] From /Users/huanghaihun/anaconda3/lib/python3.7/site-packages/tensorflow/python/data/ops/dataset_ops.py:410: py_func (from tensorflow.python.ops.script_ops) is deprecated and will be removed in a future version.
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    


In [7]:
body = model_body(input_shape, anchors, num_classes, None,
                     freeze_body=2,
                     weights_path='model_data/darknet53.weights.h5')

Create YOLOv3 model with 9 anchors and 2 classes.
Load weights model_data/darknet53.weights.h5.
Freeze the first 264 layers of total 267 layers.


In [4]:

def extends(true_class_probs):

    extend_true_class_probs = tf.concat(
                [true_class_probs, 1 - tf.reduce_sum(true_class_probs, axis=4, keepdims=True)], axis=4)
    return tf.argmax(extend_true_class_probs, axis=4)


In [5]:
def moveing_avg(variable, value, update_weight=0.05):    
    return variable.assign(variable * (1-update_weight) + update_weight*value )


def update(center, value, keys):
    z=extends(keys)
    ez = tf.cast(tf.expand_dims(z,4), value[0].dtype)
    v_shape = tf.shape(value)
    reshape_value = tf.reshape(value,[*v_shape[:-1],1 ,v_shape[-1]])
    group_bys = tf.reduce_mean(ez * reshape_value, axis=[0,1,2] )
    return moveing_avg(center, group_bys)


def update_centers():
    gap = outputs[3:6]
    keys = [x[..., 5:7] for x in [y1,y2,y3]]
    for (center, v,k) in zip(
        model.model_global.centers
        ,gap
        ,keys
        ):
        update(center, v, k)

In [9]:
from tqdm import *
outputs = None
num_epoch=10
optimizer = tf.keras.optimizers.Adam()
tf.summary.experimental.set_step(0)
val_step = 10
def train_step(image,y1,y2,y3):

    with tf.GradientTape() as tape:
        outputs = body(image)
        loss = loss_wrapper(outputs, [y1,y2,y3],anchors,num_classes )
        grads = tape.gradient(loss, body.trainable_weights)
        optimizer.apply_gradients(zip(grads, body.trainable_variables))
        return loss

def evaluate_step(image,y1,y2,y3):
    with tf.GradientTape() as tape:
        outputs = body(image)
        loss = loss_wrapper(outputs, [y1,y2,y3],anchors,num_classes )

for epoch in range(num_epoch):
    with tqdm(train_data,total=num_train/batch_size) as tbar:
        for x in train_data:
            image,y1,y2,y3 = x
            loss = train_step(image, y1, y2, y3)
            tbar.update(1)
            tbar.set_description("loss={:.3f}".format(loss))
            
    
    for x in range(val_step):
        image,y1,y2,y3 = x
        evaluate_step(image, y1, y2, y3)


        
        
        

loss=9415.045:   7%|▋         | 7/104.0 [00:30<07:12,  4.46s/it]


KeyboardInterrupt: 